In [ ]:
import scripts
import numpy as np
import random
import time
import csv
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt
from scripts import helpers

import base_scikit_learn_model as sk
import grid_generation as grid

In [ ]:
data = np.loadtxt('grid_800_400_0_0/grid_data_0_0.csv', dtype = float, delimiter = ',')
M = pickle.load(open('grid_800_400_0_0/cardinality_pickle.pkl', 'rb'))
m00 = M[0][0]
sorted_m00 = sorted(m00.items(), cmp = lambda x, y: cmp(x[1], y[1]), reverse = True)
mask = np.array(map(lambda x: m00[x] > 20, data[:, 5]))
masked_data = data[mask, :]

In [ ]:
y = masked_data[:, 5].astype(int)

label_encoder = LabelEncoder()
label_encoder.fit(y)

le_y = label_encoder.transform(y)
le_y = le_y.reshape(-1, 1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(le_y)
oh_y = oh_encoder.transform(le_y)

Y = le_y
X = masked_data[:, (1,2,3,4)]

days = lambda t: (int(t) % (86400*7))/ 86400
hours = lambda t: (int(t) % 86400) / 3600

days_v = np.array(map(days, X[:, 3])).reshape(-1, 1)
hours_v = np.array(map(hours, X[:, 3])).reshape(-1, 1)

main_x = np.hstack((X[:, (0, 1, 2)], days_v, hours_v))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=29, weights='distance', metric='manhattan')
clf.fit(main_x, Y)

pred_Y_prob = clf.predict_proba(main_x)
pred_labels = label_encoder.inverse_transform(np.argsort(pred_Y_prob, axis=1)[:,::-1][:,:3]) 
apk_arr = np.hstack((masked_data[:, -1].reshape(-1, 1), pred_labels))
apk_arr = apk_arr.astype(int)

apk_row = lambda r: helpers.apk(r[:1], r[1:])
t_apk = map(apk_row, apk_arr)
print np.mean(t_apk)

In [ ]:
reload(grid)
reload(sk)
g = grid.Grid(files_flag=True, pref = 'test')
sk_model = sk.SklearnModel(cross_validation_file='../main_cv_0.02_5.csv', test_file='../test.csv', grid = g, threshold=0)
sk_model.train()
sk_model.generate_submission_file('KNNearest_neigh_testing_submission_file.csv')

In [ ]:
grid_wise_data = [[[] for n in range(26)] for m in range(13)]
for i in range(len(test_data)):
    m, n = sk.get_grids_of_a_point((test_data[i][1], test_data[i][2]), sk_model.grid)[0]
    grid_wise_data[m][n].append(test_data[i])

predictions = []
for m in range(13):
    for n in range(26):
        if len(grid_wise_data[m][n]) == 0:
            continue
        else:
            predictions.append(sk_model.predict_grid(np.array(grid_wise_data[m][n]), m, n))

In [2]:
import grid_generation as grid
reload(grid)
import base_scikit_learn_model as sk
reload(sk)
g = grid.Grid(files_flag=True, pref = 'grid')
sk_model = sk.SklearnModel(cross_validation_file='../main_cv_0.02_5.csv', test_file='../code_test.csv', grid = g, threshold=20)
import sk_learn_models as knn
sk_model1 = knn.KNN(grid = g, threshold = 20)

In [3]:
sk_model.train_first_grid_and_predict()

Predicting the probablity of train set
[[1242364360 9988088517 4120068991 1242364360]
 [5082530812 9988088517 4067911129 5082530812]
 [3007552366 9988088517 4120068991 3007552366]
 [5002696932 9988088517 3916091580 5002696932]
 [1006316884 9988088517 4067911129 1006316884]]
0.0 training data prediction precision
Predicting the probability of test set
[[1254758593 7920940815 3421175639 1254758593]
 [5747062935 1511873045 5473477945 2685424598]
 [2776568148 3125174296 1557927427 7131826933]
 [7104264675 7965058889 5348440074 5527864210]
 [7859245214 7121847139 2797440100 7131826933]]
0.0378943367541 test data prediction precision


In [4]:
sk_model1.train_first_grid_and_predict()

Predicting the probablity of train set
[[1890025047 9988088517 4253203687 1890025047]
 [9682965513 9988088517 4251632872 9682965513]
 [7937617850 9988088517 4120068991 7937617850]
 [1231501903 4067911129 4237910912 1231501903]
 [5717403401 9988088517 4048573921 5717403401]]
0.0 training data prediction precision
Predicting the probability of test set
[[2154052852 4336423518 7382144836 2154052852]
 [7065354365 5083140375 2784881316 5083140375]
 [6803385643 1342336464 5674316040 1342336464]
 [1711675729 2970538710 9988088517 2970538710]
 [1890025047 4690524967 6198196797 1890025047]]
0.161079437476 test data prediction precision


In [ ]:
import helpers
reload(helpers)
reload(grid)
reload(sk)
g = grid.Grid(files_flag=True, pref = 'grid')
sk_model = sk.SklearnModel(cross_validation_file='../main_cv_0.02_5.csv', test_file='../code_test.csv', grid = g, threshold=7)
sk_model.train_first_grid_and_predict()

In [ ]:
import sk_learn_models as sk
from grid_generation import Grid
reload(sk)

g  = Grid(X = 800, Y = 400, xd = 200, yd = 100, pref = 'grid')

knn = sk.KNN(grid = g, threshold = 7)

In [ ]:
knn.train_first_grid_and_predict(m = 3, n = 1)

In [ ]:
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import preprocessing

In [ ]:
data = np.loadtxt('grid_800_400_200_100/grid_data_0_0.csv', delimiter=',')
X = data[:, (1, 2, 3, 4)]
Y = data[:, -1]

In [ ]:
y_enc = preprocessing.LabelEncoder()
y_enc.fit(Y)
Y_encoded = y_enc.transform(Y)

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [ ]:
from grid_generation import Grid
g = Grid(X = 800, Y = 400, xd = 200, yd = 100, pref = 'grid')
g.generateCardinalityMatrix()

In [ ]:
xgb1.fit(m_X, Y_encoded)

In [ ]:
mask = np.array(map(lambda x: g.M[0][0][x] > 20, data[:, 5]))
m_data = data[mask, :]
m_X = m_data[:, (1, 2, 3, 4)]
m_Y = m_data[:, -1]
y_enc = preprocessing.LabelEncoder()
y_enc.fit(m_Y)
Y_encoded = y_enc.transform(m_Y)

In [ ]:
len(y_enc.classes_)

In [1]:
cd scripts/

/Users/anil/Code/facebook/scripts


In [5]:
pwd

u'/Users/anil/Code/facebook/scripts'

In [6]:
import numpy as np

In [7]:
data = np.loadtxt('../split_train/1_place', delimiter = ',', dtype = float, skiprows = 1)

In [8]:
data[0]

array([  1.00001580e+09,   7.80000000e+01,   2.67164359e+00,
         5.54916026e+00,   8.03076923e+01,   2.69150000e+00,
         5.54925000e+00,   3.10000000e+01,   2.07015868e-01,
         1.58800983e-02,   1.16165919e+02,   4.28555696e-02,
         2.52177500e-04,   1.34945207e+04])

In [9]:
counts = data[:, (0, 1)]

In [10]:
counts = counts.astype(int)

In [11]:
counts[0]

array([1000015801,         78])

In [12]:
d = dict(counts)

In [13]:
d[1000015801]

78